In [6]:
from os import environ
from dotenv import load_dotenv
import pandas as pd
import boto3
from datetime import date
import psycopg2
from openai import OpenAI
import json

In [16]:
def get_db_connection():
    '''Forms AWS RDS postgres connection.'''
    try:
        conn = psycopg2.connect(dbname=environ["DB_NAME"],
                                host=environ["DB_HOST"],
                                user=environ["DB_USER"],
                                password=environ["DB_PASSWORD"],
                                port=environ["DB_PORT"])
        return conn
    except Exception as e:
        print(f'Error: Unable to form connection {e}')


def load_data():
    conn = get_db_connection()
    with conn.cursor() as curr:
        curr.execute('SELECT * FROM records r JOIN stories s ON r.story_id = s.story_id ORDER BY r.score DESC LIMIT 5;')
        data = curr.fetchall()
        df = pd.DataFrame(data)
        column_names = [desc[0] for desc in curr.description]
        df.columns = column_names
        curr.close()
    return df

In [21]:
def get_url_list():
    df = load_data()
    return df['story_url'].to_list()

In [22]:
url_list = get_url_list()
url_list

ValueError: Length mismatch: Expected axis has 0 elements, new values have 11 elements

In [17]:
df = load_data()
df.head()

,record_id,story_id,score,comments,record_time,story_id,title,author,story_url,creation_date,topic_id
0,623,38858012,1789,328,2024-01-04 14:32:40.171097,38858012,Niklaus Wirth has died,aarroyoc,https://twitter.com/Bertrand_Meyer/status/1742...,2024-01-03 18:50:12,10
1,678,38845510,1741,213,2024-01-04 14:32:40.171097,38845510,1D Pac-Man,memalign,https://abagames.github.io/crisp-game-lib-11-g...,2024-01-02 19:00:06,2
2,748,38821248,939,461,2024-01-04 14:32:40.171097,38821248,"Compare Google, Bing, Marginalia, Kagi, Mwmbl,...",882542F3884314B,https://danluu.com/seo-spam/,2023-12-31 02:32:22,10
3,627,38853706,694,292,2024-01-04 14:32:40.171097,38853706,Possible Meissner effect near room temperature...,zaikunzhang,https://arxiv.org/abs/2401.00999,2024-01-03 13:19:36,7
4,631,38852360,694,302,2024-01-04 14:32:40.171097,38852360,Maestro: A Linux-compatible kernel in Rust,Uriopass,https://blog.lenot.re/a/introduction,2024-01-03 09:59:33,8


In [11]:
sort_by_score = df.sort_values(by='score', ascending=False)
top_5_stories = sort_by_score.head(5)


,record_id,story_id,score,comments,record_time
425,426,38858012,1759,327,2024-01-04 13:02:21.358265
226,227,38858012,1754,326,2024-01-04 12:42:55.388842
27,28,38858012,1751,325,2024-01-04 12:24:12.084058
481,482,38845510,1740,213,2024-01-04 13:02:21.358265
84,85,38845510,1740,213,2024-01-04 12:24:12.084058
